In [9]:
%load_ext autoreload
%autoreload 2

In [10]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [11]:
with open("data/2020-09-08_sentences.txt") as sfile:
    sents = sfile.readlines()
sents = [x.replace("\n", "") for x in sents]

with open("data/2020-09-14_embeddings_mean.csv") as efile:
    embs = efile.readlines()
embs = [[float(y) for y in x.replace("\n", "").split(";")] for x in embs]

with open("data/2020-09-22_properties.csv") as pfile:
    props = pfile.readlines()
props = [[int(y) for y in x.replace("\n", "").split(";")] for x in props]

In [75]:
import numpy as np
from sklearn.model_selection import train_test_split

In [36]:
# example of a multi-output regression problem
from sklearn.datasets import make_regression
# create dataset
X, y = np.array(props), np.array(embs)
# summarize shape
print(X.shape, y.shape)
inputdim = X.shape[1]
outputdim = y.shape[1]
print(inputdim)
print(outputdim)

(6200, 87) (6200, 768)
87
768


In [114]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(inputdim, 800),
            nn.ReLU(),
            nn.Linear(800, 3000),
            nn.ReLU(),
            nn.Linear(3000, 1500),
            nn.ReLU(),
            nn.Linear(1500, outputdim)
        ).to(torch.device("cpu"))
        
    def forward(self, input_tensor):
            return self.model(input_tensor)
        

net = Net()     # define the network
optimizer = torch.optim.SGD(net.parameters(), lr=0.2)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

X_train, X_test, y_train, y_test = train_test_split(props, embs, test_size=0.20, random_state=42)
X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test)

In [143]:
torch.save(model.state_dict(), "model_tropcool1.pt")

In [144]:
# train the network
for t in range(2000):
    prediction = net(X_train)     # input x and predict based on x
    loss = loss_func(prediction, y_train)     # must be (1. nn output, 2. target)
    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients

In [86]:
from FImp import *

In [145]:
pred_X = net(X_test)

In [146]:
pred_X = pred_X.tolist()

In [148]:
mean_mse = []

In [149]:
for x, y in zip(y_test, pred_X):
    x = [int(b) for b in x]
    y = [int(a) for a in y]
    cur_mse = mse(list(x), list(y))
    mean_mse.append(cur_mse)

In [150]:
mean_mae = []

In [151]:
for x, y in zip(y_test, pred_X):
    x = [int(b) for b in x]
    y = [int(a) for a in y]
    cur_mae = mae(list(x), list(y))
    mean_mae.append(cur_mae)

In [152]:
print(np.mean(mean_mae))

0.002653519825268817


In [154]:
np.sqrt(np.mean(mean_mae))

0.05151232692539541

In [155]:
print(np.mean(mean_mse))

0.0027354250672043012


In [156]:
np.sqrt(np.mean(mean_mse))

0.052301291257523475

In [142]:
### 
dbg_pred = pred_X[100]
dbg_true = y_test[100]


print(mse(dbg_pred, dbg_true))
print("===============")
for x, y in zip(dbg_pred, dbg_true):
    print(str(x)+ "\t"+ str(y))

0.10816800969419017
0.01593412458896637	-0.4526073932647705
0.00203891284763813	-0.024547254666686058
-0.04798399284482002	0.5256749391555786
-0.020278871059417725	-0.026315439492464066
0.007399378810077906	0.2071359008550644
-0.02386697195470333	-0.07259775698184967
-0.028030259534716606	-0.26692408323287964
0.026407593861222267	0.4443999230861664
-0.027013685554265976	0.03854423388838768
-0.0391739159822464	0.3138985335826874
-0.0387357696890831	-0.11014329642057419
-0.005910329520702362	-0.13181835412979126
-0.01881188154220581	-0.24803292751312256
-0.03406000882387161	0.5204880237579346
-0.031688392162323	-0.07850523293018341
0.032598719000816345	0.3804513216018677
-0.01702294871211052	-0.016988497227430344
0.011253075674176216	0.1398751437664032
0.03226906806230545	0.06716612726449966
-0.02209879457950592	0.07858405262231827
-0.040170494467020035	-0.06990382820367813
0.06117072328925133	-0.4818931818008423
-0.035821035504341125	-0.018698547035455704
0.000567462295293808	0.14480142

## DRAFT 

In [12]:
import sys
sys.path.insert(0, './code')
from mlp import MLP

In [17]:
params = {}
inputdim = len(props[0])
outputdim = len(embs[0])
print(inputdim)
print(outputdim)
mlp_clf = MLP(params, inputdim, outputdim)

87
768


NameError: name 'utils' is not defined

In [ ]:
def run(properties, embeddings, k, model):

    # cross-validation
    logging.info('Training {0} with {1}-fold cross-validation'
                 .format(self.modelname, self.k))
    regs = [10**t for t in range(-5, -1)] if self.usepytorch else \
           [2**t for t in range(-1, 6, 1)]
    skf = StratifiedKFold(n_splits=self.k, shuffle=True,
                          random_state=self.seed)
    scores = []

    for reg in regs:
        scanscores = []
        for train_idx, test_idx in skf.split(self.train['X'],
                                             self.train['y']):
            # Split data
            X_train, y_train = self.train['X'][train_idx], self.train['y'][train_idx]

            X_test, y_test = self.train['X'][test_idx], self.train['y'][test_idx]

            # Train classifier
            if self.usepytorch:
                clf = MLP(self.classifier_config, inputdim=self.featdim,
                          nclasses=self.nclasses, l2reg=reg,
                          seed=self.seed)
                clf.fit(X_train, y_train, validation_data=(X_test, y_test))
            else:
                if self.classifier_config['classifier']['nhid'] == -1:
                    clf = LogisticRegression(C=reg, random_state=self.seed)
                    clf.fit(X_train, y_train)
                elif self.classifier_config['classifier']['nhid'] == 0:
                    clf = LogisticRegression(C=reg, random_state=self.seed)
                    clf.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            scanscores.append(score)
        # Append mean score
        scores.append(round(100*np.mean(scanscores), 2))

    # evaluation
    logging.info([('reg:' + str(regs[idx]), scores[idx])
                  for idx in range(len(scores))])
    optreg = regs[np.argmax(scores)]
    devaccuracy = np.max(scores)
    logging.info('Cross-validation : best param found is reg = {0} \
        with score {1}'.format(optreg, devaccuracy))

    logging.info('Evaluating...')
    if self.usepytorch:
        clf = MLP(self.classifier_config, inputdim=self.featdim,
                  nclasses=self.nclasses, l2reg=optreg,
                  seed=self.seed)
        clf.fit(self.train['X'], self.train['y'], validation_split=0.05)
    else:
        clf = LogisticRegression(C=optreg, random_state=self.seed)
        clf.fit(self.train['X'], self.train['y'])
    yhat = clf.predict(self.test['X'])

    # scores = cross_val_score(clf, self.test['X'], self.test['y'], cv=self.k)
    print("STD DEV : " + str(np.std(scores)))
    print("MEAN : " + str(np.mean(scores)))


    print("____REPORT____")
    #print("precision : " + str(precision_score(self.test["y"], yhat,  average=None)))
    #print("recall : " + str(recall_score(self.test["y"], yhat, average=None)))
    print(classification_report(self.test["y"], yhat))

    testaccuracy = clf.score(self.test['X'], self.test['y'])
    testaccuracy = round(100*testaccuracy, 2)

    return devaccuracy, testaccuracy, yhat